In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls drive/My\ Drive/proyecto_patrones/

denoised_testing   MetaData.npy    testing	      training_split.zip
denoised_training  proyecto.ipynb  testing_split.zip  Untitled0.ipynb
MetaData.mat	   readme.txt	   training


In [0]:
!unzip drive/My\ Drive/proyecto_patrones/training_split

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, Input, Dropout, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from pybalu.feature_transformation import pca, normalize
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from sklearn.preprocessing import MinMaxScaler

import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

Using TensorFlow backend.


# Data

In [0]:
def load_data(path, imgs):
  data = []
  for img in imgs:
    img_loaded = cv2.imread(path + '/' + img)
    img_loaded = cv2.resize(img_loaded, (224, 224))
    data.append(img_loaded)
  data = np.array(data)
  return data




def flip_img(imgs):
  flipped = []
  for img in imgs:
    flipped.append(img[:, ::-1])
  flipped = np.array(flipped)
  
  return np.vstack((imgs, flipped))

In [0]:
def build_dataset(pos_X, neg_X):
  pos_neg_X = np.vstack((pos_X, neg_X))
  X = []
  for img in range(0, len(pos_neg_X), 2):
    X.append(np.array([pos_neg_X[img], pos_neg_X[img + 1]]))
  X = np.array(X)
  return X

def extract_features(X, model):
  old = X[:, 0, :, :]
  new = X[:, 1, :, :]

  features_old = model.predict(old, verbose=1, batch_size=256)
  features_new = model.predict(new, verbose=1, batch_size=256)
  
  return features_old, features_new

def pca_transform(features_new, features_old):
  X_new_pca, _, A_new, Xm_new, _ = pca(features_new, n_components=10)
  X_old_pca, _, A_old, Xm_old, _ = pca(features_old, n_components=10)
  
  return X_new_pca, A_new, Xm_new, X_old_pca, A_old, Xm_old

In [0]:
pos_imgs = os.listdir('training_split/pos')
pos_imgs.sort()

neg_imgs = os.listdir('training_split/neg')
neg_imgs.sort()

pos_X = load_data('training_split/pos', pos_imgs)
neg_X = load_data('training_split/neg', neg_imgs)


y_pos = np.array([1] * (pos_X.shape[0]))
y_neg = np.array([0] * (neg_X.shape[0]))

In [0]:
X_train_pos, X_val_pos, y_train_pos, y_val_pos = train_test_split(pos_X, y_pos, test_size=0.1, shuffle=True)
X_train_neg, X_val_neg, y_train_neg, y_val_neg = train_test_split(neg_X, y_neg, test_size=0.1, shuffle=True)

In [0]:
X_train_pos = flip_img(X_train_pos)
X_train_neg = flip_img(X_train_neg)

In [0]:
X_train = build_dataset(X_train_pos, X_train_neg)
X_val = build_dataset(X_val_pos, X_val_neg)

X_train.shape, X_val.shape

((6300, 2, 224, 224, 3), (350, 2, 224, 224, 3))

# Feature

In [0]:
from keras_vggface.vggface import VGGFace

layer_name_vggface = 'fc7'

vgg_model = VGGFace(input_shape=(224, 224, 3))
out = vgg_model.get_layer(layer_name_vggface).output
vgg_model_new = Model(vgg_model.input, out)

W0616 03:47:00.975154 140347530438528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0616 03:47:00.999501 140347530438528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0616 03:47:01.008525 140347530438528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0616 03:47:01.047794 140347530438528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0616 03:47:04.640244 140347530438528 deprecation_wrapp

In [0]:
layer_name_model = 'fc2'

model = VGG16(weights='imagenet', include_top=True)
out = model.get_layer(layer_name_model).output
model_new = Model(model.input, out)

In [0]:
X_new_features_train, X_old_features_train = extract_features(X_train, vgg_model_new)
X_new_pca_train, A_new, Xm_new, X_old_pca_train, A_old, Xm_old = pca_transform(X_new_features_train, X_old_features_train)

6300/6300 [==============================] - 31s 5ms/step


In [0]:
X_new_features_val, X_old_features_val = extract_features(X_val, vgg_model_new)

X_new_pca_val = np.matmul(X_new_features_val - Xm_new, A_new)
X_old_pca_val = np.matmul(X_old_features_val - Xm_old, A_old)

350/350 [==============================] - 2s 5ms/step


In [0]:
X_features_train = np.hstack((X_new_pca_train, X_old_pca_train))
X_features_val = np.hstack((X_new_pca_val, X_old_pca_val))

X_features_train.shape, X_features_val.shape

((6300, 20), (350, 20))

In [0]:
y_train = np.array([1] * (X_features_train.shape[0] // 2) + [0] * (X_features_train.shape[0] // 2))
y_val = np.array([1] * (X_features_val.shape[0] // 2) + [0] * (X_features_val.shape[0] // 2))
y_train.shape,  y_val.shape

((6300,), (350,))

# Model

In [0]:
#clf = SVC(gamma='auto')
clf = MLPClassifier(hidden_layer_sizes=(100, 50))
clf.fit(X_features_train, y_train) 

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 50), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
clf.score(X_features_val, y_val)

0.5028571428571429

In [0]:
y_pred = clf.predict(X_features_val)
y_true = y_val

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.49      0.58      0.53       175
           1       0.48      0.39      0.43       175

    accuracy                           0.49       350
   macro avg       0.49      0.49      0.48       350
weighted avg       0.49      0.49      0.48       350



# Test

In [0]:
!unzip drive/My\ Drive/proyecto_patrones/testing_split.zip

In [0]:
pos_imgs = os.listdir('testing_split/pos')
pos_imgs.sort()

neg_imgs = os.listdir('testing_split/neg')
neg_imgs.sort()

pos_X = load_data('testing_split/pos', pos_imgs)
neg_X = load_data('testing_split/neg', neg_imgs)

In [0]:
X_test = build_dataset(pos_X, neg_X)
X_test.shape

(944, 2, 224, 224, 3)

In [0]:
X_new_features_test, X_old_features_test = extract_features(X_test, vgg_model_new)

X_new_pca_test = np.matmul(X_new_features_test - Xm_new, A_new)
X_old_pca_test = np.matmul(X_old_features_test - Xm_old, A_old)

944/944 [==============================] - 4s 5ms/step


In [0]:
X_features_test = np.hstack((X_new_pca_test, X_old_pca_test))
X_features_test.shape

(944, 20)

In [0]:
y_test = np.array([1] * (X_features_test.shape[0] // 2) + [0] * (X_features_test.shape[0] // 2))
y_test.shape

(944,)

In [0]:
clf.score(X_features_test, y_test)

0.5095338983050848

In [0]:
y_pred = clf.predict(X_features_test)
y_true = y_test

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.51      0.57      0.54       472
           1       0.51      0.45      0.48       472

    accuracy                           0.51       944
   macro avg       0.51      0.51      0.51       944
weighted avg       0.51      0.51      0.51       944

